In [1]:
import numpy as np
from collections import Counter
count=0
# --- Suffix Tree Implementation ---
class SuffixTreeNode:
    def __init__(self):
        self.children = {}  # key: char, value: (start, length, child node)
        self.suffix_index = -1  # only for leaf nodes

class SuffixTree:
    def __init__(self, text):
        self.text = text
        self.root = SuffixTreeNode()
        self.build_suffix_tree()

    def build_suffix_tree(self):
        n = len(self.text)
        for i in range(n):  # insert all suffixes
            current = self.root
            j = i
            while j < n:
                c = self.text[j]
                if c not in current.children:
                    # create new edge from j to end of string
                    child = SuffixTreeNode()
                    current.children[c] = (j, n - j, child)
                    child.suffix_index = i
                    break
                else:
                    start, length, child = current.children[c]
                    k = 0
                    while k < length and j + k < n and self.text[start + k] == self.text[j + k]:
                        k += 1
                    if k == length:
                        current = child
                        j += k
                    else:
                        # split the edge
                        split = SuffixTreeNode()
                        current.children[c] = (start, k, split)
                        split.children[self.text[start + k]] = (start + k, length - k, child)
                        new_leaf = SuffixTreeNode()
                        split.children[self.text[j + k]] = (j + k, n - (j + k), new_leaf)
                        new_leaf.suffix_index = i
                        break

# --- Suffix Array from Suffix Tree ---
def get_suffix_array_from_tree(suffix_tree):
    result = []

    def dfs(node):
        if node.suffix_index != -1:
            result.append(node.suffix_index)
        for key in sorted(node.children.keys()):
            _, _, child = node.children[key]
            dfs(child)

    dfs(suffix_tree.root)
    return result

# --- BWT & Tables ---
def compute_bwt(text, suffix_array):
    text = text + "$"
    bwt = ""
    for i in suffix_array:
        bwt += text[i - 1] if i != 0 else "$"
    return bwt

def compute_c_table(bwt):
    sorted_bwt = sorted(bwt)
    unique_chars = sorted(set(bwt))
    c_table = {char: sorted_bwt.index(char) for char in unique_chars}
    return c_table

def compute_occ_table(bwt):
    unique_chars = sorted(set(bwt))
    occ_table = np.zeros((len(unique_chars), len(bwt) + 1), dtype=int)
    char_to_index = {char: i for i, char in enumerate(unique_chars)}

    for i in range(1, len(bwt) + 1):
        occ_table[:, i] = occ_table[:, i - 1]
        char = bwt[i - 1]
        occ_table[char_to_index[char], i] += 1

    return occ_table, char_to_index

# --- Backward Search ---
def backward_search(pattern, c_table, occ_table, char_to_index, bwt):
    s, e = 1, len(bwt)
    for char in reversed(pattern):
        if char not in char_to_index:
            return []
        idx = char_to_index[char]
        s = c_table[char] + occ_table[idx, s - 1] + 1
        e = c_table[char] + occ_table[idx, e]
        if s > e:
            return []
    return list(range(s - 1, e))

# --- Main Program ---
if __name__ == "__main__":

    reference_genome = (
    "ATGCGTACGTAGCTAGCTAGGCTAGCTAGGCTACGTAGCTAGCTAGGCTAGCGTACGTAGCTAGCTAGGCTACGTAGCTAG"
)


    # Step 1: Build Suffix Tree and get Suffix Array
    tree = SuffixTree(reference_genome + "$")
    suffix_array = get_suffix_array_from_tree(tree)
    bwt_string = compute_bwt(reference_genome, suffix_array)
    c_table = compute_c_table(bwt_string)
    occ_table, char_to_index = compute_occ_table(bwt_string)

    # Step 2: Print FM-index details
    print("Suffix Array:", suffix_array)
    print("BWT String:", bwt_string)
    print("C Table:", c_table)
    print("Occurrence Table:")
    for char, idx in char_to_index.items():
        print(f"{char}: {occ_table[idx]}")

    # Step 3: Seed-based search and reconstruction
    seeds = [
    "ATG", "TGC", "GCG", "CGT", "GTA", "TAC", "ACG", "CGT", "GTA", "TAG",  # from start
    "AGC", "GCT", "CTA", "TAG", "AGG", "GGC", "GCT", "CTA", "TAG", "AGC",  # middle
    "CGT", "GTA", "TAG", "AGC", "GCT",                                     # near end
    "XYZ", "ABC", "NOP", "TTT", "GGG"                                      # do not match
]
 # You can modify these
    seed_len = 3
    reconstruction = ['-' for _ in range(len(reference_genome))]

    # Count occurrences of each seed
    seed_counts = Counter(seeds)

    print("\n Performing seed-based search and reconstruction:")
    for seed in seed_counts:
        print(f"\n Seed: '{seed}'")
        sa_indices = backward_search(seed, c_table, occ_table, char_to_index, bwt_string)
        positions = [suffix_array[i] for i in sa_indices]
        print(f" Found at positions: {positions}")

        used = 0
        max_uses = seed_counts[seed]
        for pos in positions:
            if used >= max_uses:
                break
            if pos + seed_len <= len(reference_genome):
                reconstruction[pos:pos + seed_len] = list(seed)
                used += 1

    reconstructed_query = ''.join(reconstruction)
    print("\n Reconstructed Read Skeleton from Seeds:")
    print(reconstructed_query)

Suffix Array: [81, 71, 54, 32, 6, 79, 49, 75, 58, 36, 10, 62, 23, 40, 14, 66, 27, 44, 18, 0, 51, 3, 72, 55, 33, 7, 69, 30, 77, 47, 60, 21, 38, 12, 64, 25, 42, 16, 80, 50, 2, 68, 29, 76, 46, 59, 20, 37, 11, 63, 24, 41, 15, 67, 28, 45, 19, 52, 4, 73, 56, 34, 8, 70, 53, 31, 5, 78, 48, 74, 57, 35, 9, 61, 22, 39, 13, 65, 26, 43, 17, 1]
BWT String: GTTTTTTTTTTTTTTTTTT$GGAAAAGGGGGGGGGGGGAATGGAGAGAAAAAAAAAACCCCCCCGCGCCGGGGCCCCCCCCA
C Table: {'$': 0, 'A': 1, 'C': 20, 'G': 38, 'T': 63}
Occurrence Table:
$: [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1]
A: [ 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  1
  2  3  4  4  4  4  4  4  4  4  4  4  4  4  4  5  6  6  6  6  7  7  8  8
  9 10 11 12 13 14 15 16 17 18 18 18 18 18 18 18 18 18 18 18 18 18 18 18
 18 18 18 18 18 18 18 18 18 18 19]
C: [ 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  